In [1]:
import pandas as pd

In [11]:
df=pd.read_csv('time.csv')
df.head()

,Date,store,product,number_sold
0,2010-01-01,0,0,801
1,2010-01-02,0,0,810
2,2010-01-03,0,0,818
3,2010-01-04,0,0,796
4,2010-01-05,0,0,808


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230090 entries, 0 to 230089
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Date         230090 non-null  object
 1   store        230090 non-null  int64 
 2   product      230090 non-null  int64 
 3   number_sold  230090 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 7.0+ MB


In [20]:
sample=df.sample(n=1000)

In [24]:
sample.head()

,Date,store,product,number_sold
34134,2013-06-18,1,0,674
188647,2013-07-12,5,7,899
77670,2015-09-01,2,3,787
40532,2012-12-24,1,2,719
26958,2011-10-25,0,8,667


In [43]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
sample['number_sold']=scaler.fit_transform(np.array(sample['number_sold']).reshape(-1,1))

In [44]:
look_back=980
look_ahead=20
X = []
y = []

for i in range(len(sample) - look_back - look_ahead + 1):
    X.append(sample['number_sold'].values[i:i+look_back])
    y.append(sample['number_sold'].values[i+look_back:i+look_back+look_ahead])

X = np.array(X)
y = np.array(y)

In [45]:
y

array([[0.0554939 , 0.6581576 , 0.74250832, 0.86459489, 0.62264151,
        0.60932297, 0.74139845, 0.54384018, 0.55382908, 0.67702553,
        0.78135405, 0.618202  , 0.63041065, 0.49722531, 0.74028857,
        0.69811321, 0.59711432, 0.78246393, 0.7436182 , 0.79578246]])

In [46]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
X=np.reshape(X,(X.shape[0],X.shape[1],1))
model=Sequential()
model.add(SimpleRNN(50, input_shape=(look_back, 1)))
model.add(Dense(look_ahead))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X, y, epochs=50, batch_size=1)

Epoch 1/50
1/1 [==============================] - 1s 697ms/step - loss: 0.5276
Epoch 2/50
1/1 [==============================] - 0s 61ms/step - loss: 0.4263
Epoch 3/50
1/1 [==============================] - 0s 60ms/step - loss: 0.3606
Epoch 4/50
1/1 [==============================] - 0s 62ms/step - loss: 0.3121
Epoch 5/50
1/1 [==============================] - 0s 61ms/step - loss: 0.2735
Epoch 6/50
1/1 [==============================] - 0s 60ms/step - loss: 0.2419
Epoch 7/50
1/1 [==============================] - 0s 62ms/step - loss: 0.2146
Epoch 8/50
1/1 [==============================] - 0s 62ms/step - loss: 0.1896
Epoch 9/50
1/1 [==============================] - 0s 63ms/step - loss: 0.1661
Epoch 10/50
1/1 [==============================] - 0s 58ms/step - loss: 0.1443
Epoch 11/50
1/1 [==============================] - 0s 61ms/step - loss: 0.1246
Epoch 12/50
1/1 [==============================] - 0s 63ms/step - loss: 0.1072
Epoch 13/50
1/1 [==============================] - 0s 59ms/s

In [47]:
last_sequence = np.array(sample['number_sold'][-look_back:])
last_sequence = np.reshape(last_sequence, (1, look_back, 1))
forecast = model.predict(last_sequence)

1/1 [==============================] - 0s 127ms/step


In [48]:
forecast

array([[-0.04844244,  0.4375052 ,  0.7136353 ,  0.80938977,  0.6398079 ,
         0.28070268,  0.7266433 ,  0.35992438,  0.37619466,  0.49071217,
         0.77741146,  0.7320699 ,  0.6075617 ,  0.6797605 ,  0.7394335 ,
         0.48818815,  0.2956022 ,  1.031847  ,  0.79702526,  0.99053633]],
      dtype=float32)

In [49]:
y

array([[0.0554939 , 0.6581576 , 0.74250832, 0.86459489, 0.62264151,
        0.60932297, 0.74139845, 0.54384018, 0.55382908, 0.67702553,
        0.78135405, 0.618202  , 0.63041065, 0.49722531, 0.74028857,
        0.69811321, 0.59711432, 0.78246393, 0.7436182 , 0.79578246]])

In [53]:
ytest=scaler.inverse_transform(y)

In [54]:
ypred=scaler.inverse_transform(forecast)

In [55]:
MAPE = np.abs((ytest - ypred) / ytest)
np.mean(MAPE)*100

15.32735134689581

In [80]:
result = pd.DataFrame({'Predicted': ypred.flatten(), 'True': ytest.flatten()})
result

,Predicted,True
0,210.353363,304.0
1,648.192200,847.0
2,896.985413,923.0
3,983.260254,1033.0
4,830.466919,815.0
5,506.913147,803.0
6,908.705627,922.0
7,578.291870,744.0
8,592.951355,753.0
9,696.131653,864.0
